In [3]:
import pandas as pd
import numpy as np
import glob
import laspy
import open3d as o3d
import cv2

import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from PIL import Image
import os

In [4]:
def GetPathRelations(full_path_to_data):        
    ground_removed_image_paths = []
    laz_point_cloud_paths = []
        
    # Find full path to all images
    for path in glob.glob(full_path_to_data+'/ImagesGroundRemovedLarge/*'):
        ground_removed_image_paths.append(path)
    
    # Find full path to all laz files
    for path in glob.glob(full_path_to_data+'/LazFilesWithHeightRemoved/*'):
        laz_point_cloud_paths.append(path)
            
    ground_removed_image_paths.sort()
    laz_point_cloud_paths.sort()
    assert(len(ground_removed_image_paths)==len(laz_point_cloud_paths))
    return ground_removed_image_paths, laz_point_cloud_paths

def MaxMinNormalize(arr):
    return (arr - np.min(arr))/(np.max(arr)-np.min(arr))

def CastAllXValuesToImage(arr, x_pixels):
    return (MaxMinNormalize(arr))*x_pixels

def CastAllYValuesToImage(arr, y_pixels):
    return (1-MaxMinNormalize(arr))*y_pixels

In [ ]:
matplotlib.rc('xtick', labelsize=3) 
matplotlib.rc('ytick', labelsize=3)
matplotlib.rcParams.update({'font.size': 5})

all_path_relations = GetPathRelations("/home/frederik/data/TestData/data")
path_tuples = list(zip(*all_path_relations))

image_size = 4096
network_size = int(image_size/16)
amount_of_crops = image_size//network_size


if not os.path.exists("images_labels/"+str(image_size)):
    os.makedirs("images_labels/"+str(image_size))

# Normalize to -1 and 1
transform_img_gray = transforms.Compose(
    [transforms.Resize((image_size,image_size)),
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

data = []
for path in path_tuples[1:2]:
    image_path, laz_path = path
    
    # Image to training set
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    image = np.where(image >= 0, image, 0)
    image = image/np.max(image)
    image = (image*255).astype(np.uint8)
    
    # Create pil image
    image = Image.fromarray(image)
    
    image_rotated0 = transform_img_gray(image)
    _, x_pixels, y_pixels = image_rotated0.shape
    
    # Generate labels 
    las = laspy.read(laz_path, laz_backend=laspy.compression.LazBackend.LazrsParallel)
    
    y_values = np.rint(CastAllXValuesToImage(las.X, x_pixels)).astype(np.int32)
    x_values = np.rint(CastAllYValuesToImage(las.Y, y_pixels)).astype(np.int32)
    
    powerline_mask = (las.classification == 14)
    x_powerline_values = x_values[powerline_mask]
    x_powerline_values = np.where(x_powerline_values < x_pixels, x_powerline_values, x_pixels-1)
    x_powerline_values = np.where(x_powerline_values >= 0, x_powerline_values, 0)
    
    y_powerline_values = y_values[powerline_mask]
    y_powerline_values = np.where(y_powerline_values < y_pixels, y_powerline_values, y_pixels-1)
    y_powerline_values = np.where(y_powerline_values >= 0, y_powerline_values, 0)
    
    labels = np.zeros((x_pixels, y_pixels)).astype(np.uint8)
    for i in range(len(x_powerline_values)):
        labels[x_powerline_values[i], y_powerline_values[i]] = 255
    
    lines_image = labels
    
    # Create kernel
    kernel = np.ones((3, 3), np.uint8)
    lines_image = cv2.dilate(labels, kernel, iterations=1)
    
    lines_image = Image.fromarray(lines_image)
    lines_image_rotated0 = transform_img_gray(lines_image)
    
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(8,8))
    ax0.set_title('Image')
    ax0.imshow(image_rotated0[0].numpy() ,cmap='gray')

    ax1.set_title('Generated Label')
    ax1.imshow(lines_image_rotated0[0].numpy(), cmap='gray')
    #ax0.axis('off')
    #ax1.axis('off')
    plt.savefig('images_labels/'+str(image_size)+"/"+str(image_size)+'.png', dpi = 300, bbox_inches = 'tight')    
    plt.close(fig)
    
    rects = []
    cropped_list = []
    for i in range(amount_of_crops):
        x_start_index = network_size*i
        x_end_index = network_size*(i+1)

        for j in range(amount_of_crops):
            # Generate slice indices
            y_start_index = network_size*j
            y_end_index = network_size*(j+1)

            # Apply slice mask and obtain the cropped image
            cropped_image = image_rotated0[0][x_start_index:x_end_index,y_start_index:y_end_index]
            
            # Apply slice mask and obtain the cropped label
            cropped_label = lines_image_rotated0[0][x_start_index:x_end_index,y_start_index:y_end_index]
            
            # Append
            cropped_list.append((cropped_image, cropped_label))
            
            rect1 = patches.Rectangle((y_start_index, x_start_index),
                                      network_size, network_size, linewidth=1, edgecolor='r', facecolor='none')
            rect2 = patches.Rectangle((y_start_index, x_start_index),
                                      network_size, network_size, linewidth=1, edgecolor='r', facecolor='none')
            rects.append((rect1, rect2))
            
    for number, i in enumerate(cropped_list):
        img, lab = i
        
        rect1, rect2 = rects[number][0], rects[number][1]
        
        fig, (ax0, ax1, ax2, ax3) = plt.subplots(1, 4, figsize=(7,7))
        
        ax0.set_title('Full Scaled Image')
        ax0.imshow(image_rotated0[0].numpy(), cmap='gray')
        ax0.add_patch(rect1)
        
        ax1.set_title('Patched Image')
        ax1.imshow(img ,cmap='gray')
        
        ax2.set_title('Labelled Image')
        ax2.imshow(lines_image_rotated0[0].numpy(), cmap='gray')
        ax2.add_patch(rect2)

        ax3.set_title('Patched Labels')
        ax3.imshow(lab, cmap='gray')
        
        #ax0.axis('off')
        #ax1.axis('off')
        plt.savefig('images_labels/'+str(image_size)+"/"+str(image_size)+"_"+str(number)+"_"+str(network_size)+'.png',
                    dpi = 300, bbox_inches = 'tight')
        plt.close(fig)
        
        


4095


4095


In [29]:
matplotlib.rc('xtick', labelsize=8) 
matplotlib.rc('ytick', labelsize=8)
matplotlib.rcParams.update({'font.size': 12})

all_path_relations = GetPathRelations("/home/frederik/data/TestData/data")
path_tuples = list(zip(*all_path_relations))

image_size = 992
network_size = int(image_size/16)
amount_of_crops = image_size//network_size


if not os.path.exists("images_labels/"+str(image_size)+"dilaVSnodila"):
    os.makedirs("images_labels/"+str(image_size)+"dilaVSnodila")

# Normalize to -1 and 1
transform_img_gray = transforms.Compose(
    [transforms.Resize((image_size,image_size)),
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

data = []
for path in path_tuples[1:2]:
    image_path, laz_path = path
    
    # Image to training set
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    image = np.where(image >= 0, image, 0)
    image = image/np.max(image)
    image = (image*255).astype(np.uint8)
    
    # Create pil image
    image = Image.fromarray(image)
    
    image_rotated0 = transform_img_gray(image)
    _, x_pixels, y_pixels = image_rotated0.shape
    
    # Generate labels 
    las = laspy.read(laz_path, laz_backend=laspy.compression.LazBackend.LazrsParallel)
    
    y_values = np.rint(CastAllXValuesToImage(las.X, x_pixels)).astype(np.int32)
    x_values = np.rint(CastAllYValuesToImage(las.Y, y_pixels)).astype(np.int32)
    
    powerline_mask = (las.classification == 14)
    x_powerline_values = x_values[powerline_mask]
    x_powerline_values = np.where(x_powerline_values < x_pixels, x_powerline_values, x_pixels-1)
    x_powerline_values = np.where(x_powerline_values >= 0, x_powerline_values, 0)
    
    y_powerline_values = y_values[powerline_mask]
    y_powerline_values = np.where(y_powerline_values < y_pixels, y_powerline_values, y_pixels-1)
    y_powerline_values = np.where(y_powerline_values >= 0, y_powerline_values, 0)
    
    labels = np.zeros((x_pixels, y_pixels)).astype(np.uint8)
    for i in range(len(x_powerline_values)):
        labels[x_powerline_values[i], y_powerline_values[i]] = 255
    
    lines_image = labels
    
    # Create kernel
    kernel = np.ones((3, 3), np.uint8)
    lines_image = cv2.dilate(labels, kernel, iterations=1)
    
    lines_image = Image.fromarray(lines_image)
    lines_image_rotated0 = transform_img_gray(lines_image)
    
    rects = []
    cropped_list = []
    for i in range(amount_of_crops):
        x_start_index = network_size*i
        x_end_index = network_size*(i+1)

        for j in range(amount_of_crops):
            # Generate slice indices
            y_start_index = network_size*j
            y_end_index = network_size*(j+1)

            # Apply slice mask and obtain the cropped image
            cropped_image = image_rotated0[0][x_start_index:x_end_index,y_start_index:y_end_index]
            
            # Apply slice mask and obtain the cropped label
            cropped_label = lines_image_rotated0[0][x_start_index:x_end_index,y_start_index:y_end_index]
            
            # Append
            cropped_list.append((cropped_image, cropped_label))
            
            rect1 = patches.Rectangle((y_start_index, x_start_index),
                                      network_size, network_size, linewidth=1, edgecolor='r', facecolor='none')
            rect2 = patches.Rectangle((y_start_index, x_start_index),
                                      network_size, network_size, linewidth=1, edgecolor='r', facecolor='none')
            rects.append((rect1, rect2))
            
            
    for number, i in enumerate(cropped_list):
        img, lab = i
        
        rect1, rect2 = rects[number][0], rects[number][1]
        
        plt.figure()
        plt.title("Patched Labels Dilated")
        plt.imshow(lab, cmap='gray')
        plt.savefig("images_labels/"+str(image_size)+"dilaVSnodila"+"/"+str(image_size)+"_"+str(number)+"_"+str(network_size)+"nodila"+'.png', dpi = 300)
        plt.close()
        
        #fig, (ax0, ax1, ax2, ax3) = plt.subplots(1, 4, figsize=(7,7))
        
        #ax0.set_title('Full Scaled Image')
        #ax0.imshow(image_rotated0[0].numpy(), cmap='gray')
        #ax0.add_patch(rect1)
        
        #ax1.set_title('Patched Image')
        #ax1.imshow(img ,cmap='gray')
        
        #ax2.set_title('Labelled Image')
        #ax2.imshow(lines_image_rotated0[0].numpy(), cmap='gray')
        #ax2.add_patch(rect2)

        #ax3.set_title('Patched Labels')
        #ax3.imshow(lab, cmap='gray')
        
        #ax0.axis('off')
        #ax1.axis('off')
        #plt.savefig('images_labels/'+str(image_size)+"/"+str(image_size)+"_"+str(number)+"_"+str(network_size)+'.png',
                    #dpi = 300, bbox_inches = 'tight')
        #plt.close(fig)

4095


In [48]:
# Generate labels 
las = laspy.read(path_tuples[1:2][0][1], laz_backend=laspy.compression.LazBackend.LazrsParallel)
las = las[las.HeightAboveGround < 15]

las = las[las.classification!=14]

point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))

geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)
o3d.visualization.draw_geometries([geom])


4095


In [45]:
path_tuples[1:2][0][1]

'/home/frederik/data/TestData/data/LazFilesWithHeightRemoved/PUNKTSKY_00005_1km_6146_468_height_filtered.laz'